Puisque l'on commence à être nombreux à travailler sur l'adaptation de 
domaine avec les réseaux de neurones, je propose que l'on se rassemble tous
dans la même pièce pour discuter de nos besoins communs en matière 
d'implémentation. Ainsi nous ne perdrons pas de temps à coder les mêmes choses
chacun de son coté et il sera plus facile de s'entre aider pour le débogage.

Je rassemble ici les diverses choses que l'on est plusieurs à avoir besoin 
(voire tous).


Construire le réseaux de neurones
=================================
1. Gérer les architectures séquentielles et en graphe
2. Gérer les entrées multiples dans une même couche
	1. équivalent à partager les poids entre les couches
	2. équivalent aux réseaux siamois
	3. En Lasagne cela signifie être capable de 'cloner' les couches
3. Compilation

Lasagne
-------

Avec Lasagne on peut déjà gérer les structures en DAG (Directed Acyclic Graph) très facilement à l'exception des entrées multiples dans une même couche:

~~~
 A
  \
   }->C
  /
 B  
~~~

Du coup c'est déjà bien !

**Remarques:**
Beaucoup, BEAUCOUP de structures sont possibles avec les réseaux de neurone. Mais le plus important est que cette structure **dépend des données** ce qui rend un peu plus difficile l'automatisation de leur construction.

Exemples:
- shape : suivant le nombre de descripteur de nos données mais aussi leur dimensions, (batchsize, n_features) pour les données 'classiques ou (batchsize, width, length) pour les images en nuance de gris ou encore (batchsize, chanels, width, length) pour les images en couleurs, on doit adapter les couches d'entrées voir même de sortie pour les décodeurs.
- input variable : corrolaire du problème précédent la variable d'entrée d'un input layer doit suivre les dimensions des données, T.matrix, T.tensor3 ou T.tensor4.
- suivant notre type de réseaux de neurone, XANN, DANN, CANN ou classique nous n'avons pas besoin des mêmes choses : partage des poids/clonage des couches, backpropagation multiple ou pas, etc
- Enfin la structure interne du réseau va grandement changer suivant les données : convolution ou pas, nombre de couche dense et leur nombre de neurone voir même réseaux récurrents...

Pour toute ces raisons je pense qu'il n'est pas très productif de créer un super code plein de paramètre pour construire la structure de nos réseau. D'ailleur les codes utilisant Lasagne sur internet ne le font jamais.Il y a probablement une bonne raison...

In [ ]:
import theano
import theano.tensor as T
import numpy as np
import lasagne

batchsize = 50
# Prepare Theano variables for inputs and targets
input_var = T.matrix('inputs')
src_var = T.matrix('src')
target_var = T.matrix('targets')
shape = (batchsize, 2)

# Build the layers
input_layer = lasagne.layers.InputLayer(shape=shape, input_var=input_var)
dense_layer = lasagne.layers.DenseLayer(
                input_layer,
                num_units=np.prod(shape[1:]),
                nonlinearity=None,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
reshaper = lasagne.layers.ReshapeLayer(dense_layer, (-1,) + shape[1:])
output_layer = reshaper


src_layer = lasagne.layers.InputLayer(shape=shape, input_var=src_var)


Entrées multiples / Clonage / Partage des poids
-----------------------------------------------

On est plusieurs à avoir besoin de partager certaines parties de nos réseaux entre plusieurs rétro-propagations / entrées. La partie adversarial de façon générale doit être partagé entre plusieurs entrées/chemins dans le réseaux différents.

Lasagne ne permet pas (pour des raisons de calcul de gradient ?) d'avoir plusieurs entrée possible dans une même couche. Le fonctionnement est simple. Au moment de la compilation Lasagne remonte les couches en suivant l'entrée des couches. C'est suivant ce chemin que va se passer la rétro-propagation de la fonction que l'on est en train de compiler. Au final Lasagne nous force à réfléchir en terme de rétro-propagation. Si on a plusieurs combinaisons de (entrée / sortie) alors il nous faudra une fonction compilée par combinaison (entrée /sortie).

Une solution serait de pouvoir simplement cloner une couche (ie en reconstruire une partageant les poids de la couche d'origine) et de changer l'entrée et où va la sortie.

Pour cela on peut, par exemple, se faire une petite usine à clonage :

In [ ]:
def clone_DenseLayer(layer, input_layer=None):
    if not isinstance(layer, lasagne.layers.DenseLayer):
        raise ValueError("The given layer should be a lasagne.layers.DenseLayer,"
                         "{} given".format(layer.__class__))
    else:
        if input_layer is None:
            input_layer = layer.input_layer
        return lasagne.layers.DenseLayer(input_layer,
                                        num_units=layer.num_units,
                                        nonlinearity=layer.nonlinearity,
                                        W=layer.W, b=layer.b)


# Clone factory
clonable_layers = {
    lasagne.layers.DenseLayer: clone_DenseLayer
}


def clone_layer(layer, input_layer=None):
    if any([isinstance(layer, key) for key in clonable_layers.keys()]):
        return clonable_layers[layer.__class__](layer, input_layer)
    else:
        raise NotImplementedError('{} is not a clonable layer (yet)'.format(layer.__class__))


Compilation
-----------

Concernant la compilation on doit construire :
- l'expression symbolique du coût (loss symbolic expression)
- la mise à jour des paramètres concerné pendant la rétro-propagation (update the paramters during backpropagation)
- les fonctions :
    - d'entraînement
    - de test (sort directement la précision et le coût)
    - de prédiction (sort les prédictions)
    - de sortie brut (sort les probabilité ou bien les données reconstruites/décodés)
    - d'autres ?
    
Bon ça pour le coup on a déjà plutôt bien travaillé dessus.
    

In [ ]:
def crossentropy_sgd_mom(output_layer, lr=1, mom=.9, target_var=T.ivector('target')): 
    """
    Stochastic Gradient Descent compiler with optionnal momentum.

    info: it uses the categorical_crossentropy. Should be given to a softmax layer.
    
    Params
    ------
        output_layer: the output layer from which the loss and updtaes will be computed
        lr: (default=1) learning rate.
        mom: (default=0.9) momentum.

    Return
    ------
        A dictionnary with :
            -train : function used to train the neural network
            -predict : function used to predict the label
            -valid : function used to get the accuracy and loss 
            -output : function used to get the output (exm: predict the label probabilities)
    
    Example:
    --------
    >>> funs = compiler_sgd_mom(output_layer, lr=0.01, mom=0.1)
    
    """    

    input_var = lasagne.layers.get_all_layers(output_layer)[0].input_var
    # Create a loss expression for training, i.e., a scalar objective we want
    # to minimize (for our multi-class problem, it is the cross-entropy loss):
    pred = lasagne.layers.get_output(output_layer)
    loss = T.mean(lasagne.objectives.categorical_crossentropy(pred, target_var))
    # Create update expressions for training, i.e., how to modify the
    # parameters at each training step. Here, we'll use Stochastic Gradient
    # Descent and add a momentum to it.
    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.sgd(loss, params, learning_rate=lr)
    updates = lasagne.updates.apply_momentum(updates, params, momentum=mom)

    # As a bonus, also create an expression for the classification accuracy:
    acc = T.mean(T.eq(T.argmax(pred, axis=1), target_var))
    # Compile a function performing a training step on a mini-batch (by giving
    # the updates dictionary) and returning the corresponding training loss:
    train_function = theano.function([input_var, target_var], [loss, acc], 
        updates=updates, allow_input_downcast=True)

    # Create a loss expression for validation/testing. The crucial difference
    # here is that we do a deterministic forward pass through the network,
    # disabling dropout and noise layers.
    pred = lasagne.layers.get_output(output_layer, deterministic=True)
    loss = T.mean(lasagne.objectives.categorical_crossentropy(pred, target_var))
    # As a bonus, also create an expression for the classification:
    label = T.argmax(pred, axis=1)
    # As a bonus, also create an expression for the classification accuracy:
    acc = T.mean(T.eq(label, target_var))
    # Compile a second function computing the validation loss and accuracy:
    valid_function = theano.function([input_var, target_var], [loss, acc], allow_input_downcast=True)
    # Compile a function computing the predicted labels:
    predict_function = theano.function([input_var], [label], allow_input_downcast=True)
    # Compile an output function
    output_function = theano.function([input_var], [pred], allow_input_downcast=True)

    return {
            'train': train_function,
            'predict': predict_function,
            'valid': valid_function,
            'output': output_function
           }

In [ ]:
def squared_error_sgd_mom(output_layer, lr=1, mom=.9, target_var=T.matrix('target')) : 
    """
    Stochastic Gradient Descent compiler with optionnal momentum.

    info: it uses the squared_error.
    
    Params
    ------
        output_layer: the output layer from which the loss and updtaes will be computed
        lr: (default=1) learning rate.
        mom: (default=0.9) momentum.

    Return
    ------
        A dictionnary with :
            -train : function used to train the neural network
            -predict : function used to predict the label
            -valid : function used to get the accuracy and loss 
            -output : function used to get the output (exm: predict the label probabilities)
    
    Example:
    --------
    >>> funs = squared_error_sgd_mom(output_layer, lr=0.01, mom=0.1)
    
    """    

    input_var = lasagne.layers.get_all_layers(output_layer)[0].input_var
    # Create a loss expression for training, i.e., a scalar objective we want
    # to minimize (for our multi-class problem, it is the cross-entropy loss):
    pred = lasagne.layers.get_output(output_layer)
    loss = T.mean(lasagne.objectives.squared_error(pred, target_var))
    # Create update expressions for training, i.e., how to modify the
    # parameters at each training step. Here, we'll use Stochastic Gradient
    # Descent and add a momentum to it.
    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.sgd(loss, params, learning_rate=lr)
    updates = lasagne.updates.apply_momentum(updates, params, momentum=mom)

    # As a bonus, also create an expression for the classification accuracy:
    acc = T.mean((pred - target_var)**2)
    # Compile a function performing a training step on a mini-batch (by giving
    # the updates dictionary) and returning the corresponding training loss:
    train_function = theano.function([input_var, target_var], [loss, acc], 
        updates=updates, allow_input_downcast=True)
    
    # Create a loss expression for validation/testing. The crucial difference
    # here is that we do a deterministic forward pass through the network,
    # disabling dropout and noise layers.
    pred = lasagne.layers.get_output(output_layer, deterministic=True)
    loss = T.mean(lasagne.objectives.squared_error(pred, target_var))
    # As a bonus, also create an expression for the classification:
    label = T.argmax(pred, axis=1)
    # As a bonus, also create an expression for the classification accuracy:
    acc = T.mean((pred - target_var)**2)
    # Compile a second function computing the validation loss and accuracy:
    valid_function = theano.function([input_var, target_var], [loss, acc], allow_input_downcast=True)
    # Compile a function computing the predicted labels:
    predict_function = theano.function([input_var], [label], allow_input_downcast=True)
    # Compile an output function
    output_function = theano.function([input_var], [pred], allow_input_downcast=True)
    
    return {
            'train': train_function,
            'predict': predict_function,
            'valid': valid_function,
            'output': output_function
           }

Le processus d'entraînement
===========================

1. Gérer les données
	1. Les trouver/récupérer sur le net
	2. Les charger
	3. Les transformer (afin de simuler un nouveau domaine)
	4. Les séparer en training/validation/test set
2. Boucle d'entraînement "do_n_epoch()"
	1. Gérer l'entraînement alternatif de divers jeux de données avec des 
		fonctions d'entraînement compilés différentes (rétro-propagation multiple)
	2. Gérer le pré-traitement des données avant chaque époque
	3. Sauvegarder des statistiques sur la session d'entraînement (précision, coût, etc)
3. Brancher la partie adversarial sur nos réseaux de neurones facilement/automatiquement


Charger les données
-------------------

En général je recommanderais l'utilisation des conventions de scikit-learn :
- X : les données contenant les descripteurs
- y : l'objectif (labels, valeur, etc)
Dans le cas où on doit diviser les données en 3 sous-ensembles on ajoute simplement `_train _val _test`. Mais nous avons en plus de cela la division entre les données originelles et les données transformées voir même des données supplémentaires :

`y_origin` par exemple qui indique dans un `X_melanger` où on a mélangé les données originelles et les données transformées.

Bref. C'est le bordel !

Il nous faut donc une méthode pour travailler avec tous ces jeux de données sans se perdre dans des noms à la java style : `X_train_transform_non_mélangé`

Une possibilité :
puisque l'on va toujours diviser les données en 3x2 (train, val, test + X, y) autant rassembler ces données sous la même variable avec un dictionnaire.

Enfin je pense qu'on peut mettre tout ça dans un package python `datasets` avec des fonctions `load_smth()` qui renvoit les données préformattées comme on veut.

In [ ]:
import numpy as np
from sklearn.datasets import make_moons

def load_moons(noise=0.05, n_samples=500, batchsize=32):
    """
    Load the Moon dataset using sklearn.datasets.make_moons() function.

    Params
    ------
        noise: (default=0.05) the noise of the moon data generator
        n_samples: (default=500) the total number of points generated
        batchsize: (default=32) the dataset batchsize
    
    Return
    ------
        source_data: dict with the separated data

    """
    X, y = make_moons(n_samples=n_samples, shuffle=True, noise=noise, random_state=12345)
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.int32)
    
    #X, y = shuffle_array(X, y)  # Usefull ?

    n_train = int(0.4*n_samples)
    n_val = int(0.3*n_samples)+n_train

    X_train, X_val, X_test = X[0:n_train], X[n_train:n_val], X[n_val:]
    y_train, y_val, y_test = y[0:n_train], y[n_train:n_val], y[n_val:]
    
    source_data = {
                    'X_train': X_train,
                    'y_train': y_train,
                    'X_val': X_val,
                    'y_val': y_val,
                    'X_test': X_test,
                    'y_test': y_test,
                    'batchsize': batchsize,
                    }
    return source_data

Transformer les données
-----------------------

Maintenant que l'on peut gérer les données proprement, les transformer ne devrait pas être trop compliqué.
Cependant il serait domage de faire n'importe quoi.

Un "transformer" doit se présenter sous la forme d'une fonction:
~~~
data_transformed = do_smth(data)
~~~

In [ ]:
def rotate_data(X, angle=35.):
    """Apply a rotation on a 2D dataset.
    """
    theta = (angle/180.) * np.pi
    rotMatrix = np.array([[np.cos(theta), -np.sin(theta)], 
                             [np.sin(theta),  np.cos(theta)]])
    X_r = np.empty_like(X)
    X_r[:] = X[:].dot(rotMatrix)
    return X_r

def rotate_dataset(source_data, angle=35.):
    """
    Transform the given dataset by applying a rotation to it.

    target_data <- source_data . Rotation_Matrix

    Can be used only on 2D datasets !
    
    Params
    ------
        source_data: a dataset (dict with the separated data)

    Return
    ------
        target_data: dict with the separated transformed data

    """

    X_train = source_data['X_train']
    y_train = source_data['y_train']
    X_val = source_data['X_val']
    y_val = source_data['y_val']
    X_test = source_data['X_test']
    y_test = source_data['y_test']
    batchsize = source_data['batchsize']

    target_data = {
                'X_train': rotate_data(X_train, angle=angle),
                'y_train': y_train,
                'X_val': rotate_data(X_val, angle=angle),
                'y_val': y_val,
                'X_test': rotate_data(X_test, angle=angle),
                'y_test': y_test,
                'batchsize': batchsize,
                }

    return target_data



In [ ]:
source_data = load_moons()
rotate_data = rotate_dataset(source_data)

Entraînement
------------

1. Gérer l'entraînement alternatif de divers jeux de données avec des 
    fonctions d'entraînement compilés différentes (rétro-propagation multiple)
2. Gérer le pré-traitement des données avant chaque époque
3. Sauvegarder des statistiques sur la session d'entraînement (précision, coût, etc)


In [ ]:
class Trainer(object):
    def __init__(self, funs, name='trainer'):
        super(Trainner, self).__init__()
        self.name = name
        # Add the compiled functions to the object
        # by adding dynamic property to this object
        self.__dict__.update(funs)
    
    def preprocess(self, *args, **kwargs):
        pass


def training(trainers, train_data, testers=[], test_data=[], num_epochs=20, logger=None):
    """
    TODO : Explain the whole function

    Params
    ------
        trainers: list of Trainer
        train_data: list of dataset
        testers: (default=[]) list of Trainer
        test_data: (default=[]) list of datasets
        num_epochs: (default=20)
        logger: (default=None)

    Return
    ------
        stats: dict with stats
    """
    if logger is None:
        logger = new_logger()

    logger.info("Starting training...")
    final_stats = {}
    final_stats.update({trainer.name+' training loss': [] for trainer in trainers})
    final_stats.update({trainer.name+' training acc': [] for trainer in trainers})
    final_stats.update({trainer.name+' valid loss': [] for trainer in trainers})
    final_stats.update({trainer.name+' valid acc': [] for trainer in trainers})
    final_stats.update({tester.name+' valid loss': [] for tester in testers})
    final_stats.update({tester.name+' valid acc': [] for tester in testers})

    for epoch in range(num_epochs):
        # Prepare the statistics
        start_time = time.time()
        stats = { key:[] for key in final_stats.keys()}

        # Do some trainning preparations :
        for data, trainer in zip(train_data+test_data, trainers+testers):
            trainer.preprocess(data, trainer, epoch)

        # Training : (forward and backward propagation)
        # done with the iterative functions
        batches = tuple(iterate_minibatches(data['X_train'], data['y_train'], data['batchsize'], shuffle=True) 
                        for data in train_data)
        for minibatches in zip(*batches):
            for batch, trainer in zip(minibatches, trainers):
                # X, y = batch
                loss, acc = trainer.train(*batch)
                stats[trainer.name+' training loss'].append(loss)
                stats[trainer.name+' training acc'].append(acc*100)
        
        # Validation (forward propagation)
        # done with the iterative functions
        batches = tuple(iterate_minibatches(data['X_val'], data['y_val'], data['batchsize']) 
                        for data in train_data+test_data)
        for minibatches in zip(*batches):
            for batch, valider in zip(minibatches, trainers+testers):
                # X, y = batch
                loss, acc = valider.valid(*batch)
                stats[valider.name+' valid loss'].append(loss)
                stats[valider.name+' valid acc'].append(acc*100)
        
        logger.info("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        for stat_name, stat_value in sorted(stats.items()):
            if stat_value:
                mean_value = np.mean(stat_value)
                logger.info('   {:30} : {:.6f}'.format(
                    stat_name, mean_value))
                final_stats[stat_name].append(mean_value)

    return final_stats


In [ ]:
corrector_trainner = Trainer(squared_error_sgd_mom(output_layer, lr=label_rate, mom=0, target_var=target_var), 
                             'corrector',)


Brancher l'adversarial
----------------------

Cette partie est toujours présente/possible dans lnos réseaux donc il serait plutôt bien d'avoir un méthode automatique et facile.



In [ ]:

def adversarial(layers, hp_lambda=1, lr=1, mom=.9):
    """
    Stochastic Gradient Descent adversarial block compiler with optionnal momentum.

    info: it uses the categorical_crossentropy.
    
    Params
    ------
        lr: (default=1) learning rate.
        mom: (default=0.9) momentum.

    Return
    ------
        compiler_function: a function that takes an output layer and return
            a dictionnary with :
            -train : function used to train the neural network
            -predict : function used to predict the label
            -valid : function used to get the accuracy and loss 
            -output : function used to get the output (exm: predict the label probabilities)
    
    Example:
    --------
    TODO
    """    

    concat = lasagne.layers.ConcatLayer(layers, axis=0)
    rgl = ReverseGradientLayer(concat, hp_lambda=hp_lambda)
    output_layer = lasagne.layers.DenseLayer(
                    rgl,
                    num_units=len(layers),
                    nonlinearity=lasagne.nonlinearities.softmax,
                    )

    input_vars = [lasagne.layers.get_all_layers(layer)[0].input_var for layer in layers]
    true_domains = [np.ones(lasagne.layers.get_all_layers(layer)[0].shape[0], dtype=np.int32)*i 
                        for i, layer in enumerate(layers)]
    true_domains = np.hstack(true_domains)
    
    # Create a loss expression for training, i.e., a scalar objective we want
    # to minimize (for our multi-class problem, it is the cross-entropy loss):
    pred = lasagne.layers.get_output(output_layer)
    loss = T.mean(lasagne.objectives.categorical_crossentropy(pred, true_domains))
    # Create update expressions for training, i.e., how to modify the
    # parameters at each training step. Here, we'll use Stochastic Gradient
    # Descent and add a momentum to it.
    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.sgd(loss, params, learning_rate=lr)
    updates = lasagne.updates.apply_momentum(updates, params, momentum=mom)

    # As a bonus, also create an expression for the classification accuracy:
    acc = T.mean(T.eq(T.argmax(pred, axis=1), true_domains))
    # Compile a function performing a training step on a mini-batch (by giving
    # the updates dictionary) and returning the corresponding training loss:
    train_function = theano.function(input_vars, [loss, acc], 
        updates=updates, allow_input_downcast=True)

    # Create a loss expression for validation/testing. The crucial difference
    # here is that we do a deterministic forward pass through the network,
    # disabling dropout and noise layers.
    pred = lasagne.layers.get_output(output_layer, deterministic=True)
    loss = T.mean(lasagne.objectives.categorical_crossentropy(pred, true_domains))
    # As a bonus, also create an expression for the classification:
    label = T.argmax(pred, axis=1)
    # As a bonus, also create an expression for the classification accuracy:
    acc = T.mean(T.eq(label, true_domains))
    # Compile a second function computing the validation loss and accuracy:
    valid_function = theano.function(input_vars, [loss, acc], allow_input_downcast=True)
    # Compile a function computing the predicted labels:
    predict_function = theano.function(input_vars, [label], allow_input_downcast=True)
    # Compile an output function
    output_function = theano.function(input_vars, [pred], allow_input_downcast=True)

    funs = {
            'train': train_function,
            'predict': predict_function,
            'valid': valid_function,
            'output': output_function
           }

    return funs



Visualisation / Validation / Diagnostiques
==========================================

1. Courbe d'apprentissage
	1. Précision
	2. Coût
	3. Erreur de reconstruction
	4. etc
2. Matrices de confusion
3. Réseaux de neurones
	1. Poids
	2. Détecter la saturation ?
	3. Architecture
	4. etc
4. Données
	1. 2D scatter plots
	2. Exemples d'image  (avant / après transformation / correction)
	3. TSNE
	4. etc


Conventions
----------

On va tracer des graphiques. Pyplot peut être assez lourds puisqu'on peut tout contrôler. L'objectif c'est d'avoir des petites foncitons d'aide pour tracer des information qu'on veux très souvent.

Pyplot ne peut pas merge les figures mais on peut placer des axes dans une figure.
Donc le mieux est de prendre `ax=None` comme argument optionnel pour pouvoir éventuellement placer le graph dans une sous parti d'une figure.

In [ ]:
import matplotlib.pyplot as plt

def plot_curve(stats, ax=None, label=None):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.get_figure()
    # Plot learning accuracy curve
    ax.plot(stats, label=label)
    return fig, ax


def add_legend(ax, xlabel='', ylabel='', title=''):
    """Add legend to the given axes
    """
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)            

In [ ]:
%matplotlib inline

In [ ]:
stats = np.random.randn(100)+5*np.cos(np.linspace(0,10, num=100))
fig, ax = plot_curve(stats, label='Curve')
add_legend(ax, xlabel='X', ylabel='Y', title='Some Test')
plt.show()

In [ ]:
def plot_img_samples(datasets, n_sample=4, cmap='Greys_r'):
    n_datasets = len(datasets)
    # Plot some sample images:
    fig = plt.figure()
    rand = np.random.RandomState()
    for n in range(n_sample):
        i = rand.randint(source_data['X_test'].shape[0])
        for j, data in enumerate(datasets):
            sample = data['X_test'][i]
            ax = fig.add_subplot(n_sample, n_datasets, n*n_datasets+1+j)
            ax.axis('off')
            ax.imshow(sample, cmap=cmap)
            if 'name' in data:
                ax.set_title(data['name'])
    return fig, fig.get_axes()

In [ ]:
import sys, os
sys.path.append('..')
from datasets.mnist import load_mnist
import seaborn as sns; sns.set()

In [ ]:
mnist_data = load_mnist()

In [ ]:
fig, axes = plot_img_samples([mnist_data, mnist_data, mnist_data])
fig.suptitle('Image samples')
plt.show()

In [ ]:
def plot_mat(mat, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.get_figure()
    sns.heatmap(mat, cmap=plt.cm.coolwarm, ax=ax)
    return fig, ax

In [ ]:
plot_mat(np.arange(5*5).reshape(5,5))
plt.show()

In [ ]:
plot_mat(mnist_data['X_train'][0]);

In [ ]:
fig, ax = plot_mat(dense_layer.W.get_value())
add_legend(ax, xlabel='XXX')

In [ ]:
import re

def plot_learning_curve(stats, regex='acc', title=''):
    keys = [k for k in stats.keys() if re.search(regex, k)]
    print(keys)
    fig, ax = plt.subplots()
    for k in keys:
        # Plot learning accuracy curve
        ax.plot(final_stats[k], label=k)
    add_legend(ax, xlabel='epoch', ylabel='loss')
    fig.suptitle(title)

In [ ]:
final_stats = {}
final_stats['corrector valid loss'] = np.random.randn(100)+5*np.cos(np.linspace(0,10, num=100))
final_stats['blabla valid loss'] = np.random.randn(100)+3*np.cos(np.linspace(0,10, num=100))
final_stats['bloublou valid loss'] = np.random.randn(100)+1*np.cos(np.linspace(0,10, num=100))

plot_learning_curve(final_stats, regex='loss')